<a href="https://colab.research.google.com/github/Mayo-Radiology-Informatics-Lab/MIDeL/blob/main/chapters/17B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/Mayo-Radiology-Informatics-Lab/MIDeL/blob/main/chapters/img/logo.png?raw=true" alt="HOPPR logo
" width="100%">


# Obtaining Consistent LLM Outputs: From Chaos to Clarity

### Introduction
In this notebook, we will describe how to use the Instructor and Pydantic models to help increase the consistency of Large Language Model (LLM) output, which should increase the usefulness and accuracy.

The main goal of this notebook is to provide you with a step-by-step guide on how to improve data consistency when working with LLMs. We will specifically focus on using the Pydantic and Instructor packages to validate JSON responses from an LLM.

`instructor` is a lightweight Python library that provides a convenient wrapper around the client of the OpenAI compatible servers, adding validation of JSON responses from an LLM. Instructor uses the Pydantic library, which allows users to specify models for JSON schemas and data validation, ensuring that LLM responses adhere to the defined schema.


#### Key Features
- **Easy integration** Seamlessly integrates with several LLMs beyond OpenAI. See:
    - Working with different providers: https://python.useinstructor.com/
    - Examples: https://python.useinstructor.com/learning/
- **Data validation**: Ensure the JSON response from a LLM meets the specified schema. See:
    - https://docs.pydantic.dev/latest/
- **Retry Management**: Retries with error guidance if the LLM returns invalid responses. You can set the maxium number of retries.
- **Streaming Support**: Work with Lists and Partial responses effortlessly



#### Concept
<img src="https://github.com/Mayo-Radiology-Informatics-Lab/MIDeL/blob/main/chapters/img/17b.jpg?raw=true" alt="Concept Image" width="100%">



By using the Instructor package, you can have full control over agent flows without relying on complex agent frameworks. It serves as a starting point for building your own agents and ensures that the responses from LLMs are consistent and conform to the defined schema.

In the next sections, we will walk through the steps involved in enhancing data consistency using Pydantic models and the Instructor package. We will cover topics such as port forwarding, installation, creating the client, defining the response model, prompting, and more.

Let's dive in and explore the power of Pydantic models and the Instructor package in achieving data consistency in language model applications!

## Step 0: Create a LLM server with ollama

To run this notebook, we need to have a OpenAI Compatible server. You can connect you own OpenAI account, huggingface CLI or use a local server. In the next cell, we will create an LLM server running on colab so that you dont' need to use any of the prior options.
> Note: If you are running this code on the Google Colab, be sure to check if you have a GPU (Runtime menu->`Change runtime type`->`gpu T4`).

In [1]:
# Download and install Ollama which will serve the LLM
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


You can choose your desired model from https://ollama.com/library. In this notebook, we will use llama3.2 as an example.

In [2]:
# Importing nesseracy libraries
import subprocess
import time

# Start ollama in the background and use llama3.1 model

# Start the process in the background
server = subprocess.Popen(['ollama', 'serve'])
time.sleep(60) # To make sure ollama is ready in subsequent cell if you are running all not cell at a time

# To kill the server
# server.kill()

# To see all the models available: https://ollama.com/library
MODEL = 'llama3.2'
llama3 = subprocess.Popen(['ollama', 'run', MODEL])
time.sleep(90) # Make sure ollama is ready in subsequent cell if you are running all not cell at a time

# To kill the llama3
# llama3.kill()

In [4]:
import rich
# show which model(s) ollama is serving
process = subprocess.run(['ollama', 'list'], capture_output=True, text=True)
print(process.stdout)

# Check if the MODEL is in the output
if MODEL in process.stdout:
    rich.print(f"Model '{MODEL}' is successfully running.")
else:
    rich.print(f"Model '{MODEL}' is not currently running.")

NAME               ID              SIZE      MODIFIED      
llama3.2:latest    a80c4f17acd5    2.0 GB    2 minutes ago    



Model 'llama3.2' is successfully running.

### Installation


To install the required packages, run the following command in your terminal:

In [5]:
!pip install instructor pydantic rich PyYAML tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.6/345.6 kB 21.7 MB/s eta 0:00:00
  Attempting uninstall: jiter
    Found existing installation: jiter 0.9.0
    Uninstalling jiter-0.9.0:
      Successfully uninstalled jiter-0.9.0


In [6]:
# Importing the required libraries
import json
from pydantic import BaseModel, Field
from pydantic.config import ConfigDict
from typing import List, Literal, Optional, Any, Dict

import instructor
from openai import OpenAI

---
## Step 1: Prompting A *Standard* Client

Ready to see how a standard client handles things? As a benchmark, let's set up a regular OpenAI client and give it the task of extracting specific information from an interventional radiology report. We'll see how it performs before we introduce our enhanced approach!

### Configurations
In the following code block, we will set up our configuration settings.


In [7]:
# Configure the main settings for the LLM interaction.
HOST = "http://localhost:11434" # The base URL for the LLM API endpoint. The default is for Ollama running locally.

API_KEY = "ollama" # The API key for authentication with the LLM service. This is often required, but may be unused depending on the service (e.g., for local Ollama).

MODEL = MODEL # The specific language model to be used for generating responses.

SEED = 42 # A random seed for reproducibility of model outputs.

TEMP = 0.0 # The temperature setting for controlling the randomness of the model's output.

MAX_RETRIES = 5 # The maximum number of attempts the instructor library will make to validate the response from the LLM if the initial response is invalid.

It is time to create the client. `client` is responsible to contact the LLM server we have and return the response. In this notebook we are using OpenAI compatible server/clients. In case you are using `ollama`, it won't change the process.

In [8]:
# Create the client
standard_client = OpenAI(
    base_url=f"{HOST}/v1",
    api_key=API_KEY,  # required, but unused
)

Now that our client is configured, in the following code blocks, we will craft the prompts necessary to instruct the language model. We will define both a system prompt, which sets the overall context and role of the model, and a user prompt, which contains the specific task and guidelines for extraction. Following that, we will utilize a sample radiology report to test the model's ability to extract the desired information based on our defined prompts.


In [9]:
# A sample radiology report for testing the model
sample_interventional_report = """
EXAM:  MR PELVIS ABLATION, US ASSISTED GUIDANCE
  PROCEDURE: Percutaneous cryoablation of left seminal vesicle for recurrent  prostate cancer
  PRE-PROCEDURE: Patient seen and evaluated. Allergies, pertinent medications, and  history reviewed.
  Discussed risks, benefits, alternatives for procedure, and  obtained informed consent.
  Patient understands information and questions  answered. Immediately prior to starting the procedure, in the presence of the  assisting personnel,
  procedural pause was conducted to verify correct patient  identity and verification of procedure to be performed, and as applicable,
  correct side and site, correct patient position, availability of implants,  special equipment, or special requirements, and all image and specimen  identification data.
  The roles and responsibilities of care team members,  residents, and fellows were discussed.
  General anesthesia and/or sedation  provided by Anesthesiology.

  TECHNIQUE: Using sterile technique, combined ultrasound and MRI guidance, and  general anesthesia provided by the department of anesthesiology,
  percutaneous  cryoablation of left seminal vesicle was performed using 3 IceRod cryoneedles  for recurrent prostate cancer.

  FINDINGS: Patient was brought into the MR anteroom where anesthesia was  initiated.
  Patient was transferred to the MR table in a supine feet-first  position. Patient was then brought into the MR suite and the legs were  positioned in a semi-frogleg position.
  Perineum was sterilely draped and  prepped. Transperineal guidance grid was placed against the perineum.
  Cryoneedles were tested. Under direct US guidance, Abbocath cryoneedle guides  were placed and cryoneedles were placed into the recurrent prostate cancer.
  Position was confirmed with MR imaging. Saline infusion needle placed. Freezing  was initiated. A total of three freeze-thaw cycles were performed.
  During  freezing, iceball growth monitored with continuous MR imaging. After the final  freeze-thaw cycle, the needles were removed.  Dynamic contrast enhanced images  were performed.
  These demonstrated good coverage of the prostate cancer. Patient  was brought out of the MR suite and awoke. Patient tolerated the procedure well  with no immediate complications.
  Anesthesia: General Anesthesia  Imaging Guidance: Combined MR/US
  Approach: Standard  Adjunctive Procedures: saline displacement of the rectum
  Ablation Parameters: 3 Freeze-thaw cycles  Probe Removal: none
  COMPLICATIONS: none
  POST-PROCEDURE TECHNICAL EVALUATION: Zone of ablation encompassed tumor(s)  completely.
  IMPRESSION:  Successful image-guided cryoablation of left seminal vesicle for  recurrent prostate cancer.
  Follow-up imaging recommended specifically 6 month  followup with prostate MRI, PSA, clinic visit.
"""

In [10]:
# Test prompts
test1_system_prompt = "You are an expert in extracting data from radiology reports with 20 years of experience."
test1_user_prompt = """
Extract data elements from the MR guided ablation report in the <report> tag:

    Guidelines:
    - Focus on findings at the time of scan, not previous ones.
    - If information is not mentioned, use 'Not Specified'.
    - Ignore irrelevant information.
    - Use only the provided output format.
    - Expand abbreviations: sv (seminal vesicle), uvj (urethro vesicular junction), vuj (vesico urethral junction), VM (vascular malformation), US (ultrasound), LN (lymph node), CT (computed tomography), MRI (magnetic resonance imaging).

    only return your answer in this json object. Include these keys in your response:
    - organ: Extract the organ where the ablation was performed. Indicate 'Not Mentioned' if not specified. Use the provided dictionary to expand abbreviations.
    - location: the exact anatomical location of the tissue
    - tissueType: Specify the tissue type ablated: 'Muscle', 'Nerve', 'Fat', 'Ligament', 'Tendon', 'Cartilage', 'Bone', or 'Not Mentioned'. You can choose mmultuple tissues.
    - complications: Specify whether complications occurred: 'Yes', 'No', or 'Not Mentioned'
    """

In [11]:
# Let's ask the model!

# Creating the conversation for the model to pass report and instructions
messages1 = [
        {"role": "system", "content": test1_system_prompt},
        {"role": "user", "content": f"{test1_user_prompt} \n <report> {sample_interventional_report} </report>"}
    ]


# Asking the model to extract the requested information
resp1 = standard_client.chat.completions.create(
        model=MODEL,
        messages=messages1,
        temperature=TEMP,
        seed=SEED,
).choices[0].message.content


rich.print('Output:', resp1)

Output: ```
{
  "organ": "Seminal Vesicle",
  "location": "Left Seminal Vesicle",
  "tissueType": ["Muscle", "Fat"],
  "complications": "No"
}
```

Is the response a valid JSON? Let's check!

In [12]:
# Let's validate our response

try:
    json.loads(resp1)
    print("response is a valid JSON object.")
except json.JSONDecodeError:
    print("response is NOT a valid JSON object.")

response is NOT a valid JSON object.


We clearly asked the LLM to give us an answer in json format, but it didn't! And every time that you run you query there is no guarantee that you get the same response structure. Therefore, we need another tools to force LLM and make sure always get a similar response structure.

---
## Step 2: Prompting A *Patched* Client


In this notebook, we are focusing on a simple question answering (QA) task. For additional use cases, refer to [the cookbooks](https://python.useinstructor.com/examples/). to access to various examples demonstrating how to use Instructor in different scenarios.

As a simple test example, let's prompt the LLM to extract specific pieces of information from text we supply. We will then compare its answer with what we know to be the answer.
In the next cell, we write a test `system prompt` (which sets the personality or backstory of our LLM instance) and a test `user prompt` (which is the main task or request we are making, including guidelines of how to create or format the answer).


In [13]:
# Ceate a patched client using instructor
patched_client = instructor.from_openai(
    OpenAI(
        base_url=f"{HOST}/v1",
        api_key=API_KEY,  # required, but unused
    ),
    # mode: for more information: https://python.useinstructor.com/concepts/patching/
    mode=instructor.Mode.JSON,
)

#### Pydantic Models
Pydantic models are classes that inherit from pydantic.BaseModel. They offer several key benefits:

- **Data Validation**: Models automatically validate input data, ensuring that it conforms to the defined field types and constraints.
- **Type Hinting**: Models leverage Python's type annotations, providing clear type information for fields.
- **Serialization**: Models can easily convert to and from JSON, making them ideal for API development.
- **Schema Generation**: Pydantic can automatically generate JSON schemas from models, useful for documentation and API specifications.


To create a Pydantic model, simply define a class that inherits from `BaseModel`. In the next code block, fields can be customized using the `Field` function. We are also using `typing` package. With the combination of these two packages, we can force the LLM to only response in the desired format:
- `str`: Free from response. There is no limitation for the model. Although we can use max_length to limit the field.
- `Literal`: Imagine that is similar to multiple choice question. LLM can only choose one of them.
- `List`: LLM would return multiple objects in a list. We are using `List` in tandem with `Literal` to force LLM return in a specific terminology, like checking the checkboxes.

    > **Note:** the term 'model' is used heavily in AI. When we refer to Pydantic 'model' we do not mean an AI or LLM model. Instead, it means a model of how the data should be represented.

In [14]:
# Defining a simple "ProstateModel" response model to understand the pydantic models

class ProstateModel(BaseModel):
    # Each attribute has a description that will be used by the model to generate the response
    organ: str = Field(...,
        description="Extract the organ where the ablation was performed. Indicate 'Not Specified' if not specified. Use the provided dictionary to expand abbreviations."
    )
    location: str = Field(...,
        description="Extract the specific anatomical location within the organ where the ablation was performed. Indicate 'Not Mentioned' if not specified. Use the provided dictionary to expand abbreviations."
    )
    tissueType: List[Literal['Muscle', 'Nerve', 'Fat', 'Ligament', 'Tendon', 'Cartilage', 'Bone', 'Not Specified']] = Field(...,
        description="Specify the ablated tissue type. You can choose multiple tissues"
    )
    complications: Literal['Yes', 'No', 'Not Specified'] = Field(...,
        description="Specify whether complications occurred."
    )

    # OPTIONAL: We can include an example in the pydantic model. Therefore our LLM would have behave like a FewShot classification task.
    model_config = ConfigDict(
        json_schema_extra={
        'examples':
            [
                {
                    "organ": "Liver",
                    "location": "Dome",
                    "tissueType": ["Bone"],
                    "complications": "No",
                }
            ]
        }
    )

Now that we have a test response model, let's ask the LLM again, but also give it this model for its reponse, so we also remove the response structure from the `user_prompt`. Note the 'response_model' parameter in the client.chat.completions.create function call. It was 'str' before, meaning it coul dbe any legal string value. By using TestModel as the response_model, the LLM will respond in a way that conforms to the TestModel. This time it should work!

In [15]:
# Test prompts
test2_system_prompt = "You are an expert in extracting data from radiology reports with 20 years of experience."
test2_user_prompt = """
Extract data elements from the MR guided ablation report in the <report> tag:

    Guidelines:
    - Focus on findings at the time of scan, not previous ones.
    - Ignore irrelevant information.
    - Use only the provided output format.
    - Expand abbreviations: sv (seminal vesicle), uvj (urethro vesicular junction), vuj (vesico urethral junction), VM (vascular malformation), US (ultrasound), LN (lymph node), CT (computed tomography), MRI (magnetic resonance imaging).
    """

In [16]:
# Let's ask the model

# Creating the conversation for the model to pass report and instructions
messages2 = [
        {"role": "system", "content": test2_system_prompt},
        {"role": "user", "content": f"{test2_user_prompt} \n <report> {sample_interventional_report} </report>"}
]

# Asking the model to extract the requested information
resp2 = patched_client.chat.completions.create(
        model=MODEL,
        response_model=ProstateModel,
        messages=messages2,
        temperature=TEMP,
        seed=SEED,
        max_retries=MAX_RETRIES,
).model_dump_json(indent=4)


rich.print('Output:', resp2)

Output: {
    "organ": "Seminal Vesicle",
    "location": "Left Seminal Vesicle",
    "tissueType": [
        "Not Specified"
    ],
    "complications": "No"
}

In [17]:
# Let's validate our response

try:
    json.loads(resp2)
    rich.print("response is a valid JSON object.")
except json.JSONDecodeError:
    rich.print("response is NOT a valid JSON object.")

response is a valid JSON object.

Do you see the differences? By providing a structure to the `response_model` and  `max_retries` parameters, we are able to extract the requested information from the text and present it in a structured format.

---
## Step 3: A Complex Extraction



Now, let's tackle a more challenging extraction scenario often encountered in diagnostic radiology: analyzing a chest X-ray (CXR) report. These reports frequently contain intricate findings and subtle details that require a more sophisticated approach to capture accurately. To handle this complexity, we will leverage the power of nested Pydantic models. By structuring our validation models in a hierarchical way, we can represent the relationships between different findings, such as specific abnormalities within a particular lung zone or characteristics of a nodule. This allows us to extract a richer, more detailed, and highly structured representation of the information contained within the CXR report, moving beyond simple flat data extraction to a more nuanced understanding.

Let's create a tree structure using Pydantic models to represent the findings from a CXR report. This will involve nesting models to capture the hierarchical nature of the data.

Here's the conceptual tree:
```
CXRExtraction
├── airspace_opacity
│   ├── consolidation
│   │   └── modifiers
│   └── ground_glass
│       └── modifiers
└── lung_nodules
    ├── nodule
    │   ├── distribution
    │   └── modifiers
    └── mass
        ├── distribution
        └── modifiers
```

Let's implement the tree and create a pydantic model.

In [18]:
# -------------------------------------------------------------------
# AIRSPACE OPACITY COMPONENTS
# -------------------------------------------------------------------

class OpacityModifiers(BaseModel):
    """
    Attributes that describe contextual characteristics of a radiographic finding.
    """
    lung_lobe: Optional[Literal[
        "Right Upper", "Right Middle", "Right Lower",
        "Left Upper", "Left Lower", "Not Specified"
    ]] = Field(
        default=None,
        description="Select the lung lobe where the finding is located."
    )

    chronicity: Optional[Literal[
        "Acute", "Subacute", "Chronic", "Not Specified"
    ]] = Field(
        default=None,
        description="Specify the time course of the finding."
    )

class Consolidation(BaseModel):
    """
    Alveolar filling (e.g., pneumonia) visualized on chest imaging.
    """
    present: bool = Field(...,
        description="Indicate if consolidation is present."
    )
    extent: Optional[str] = Field(
        default=None,
        description="Describe the extent of consolidation (e.g., lobar, segmental)."
    )
    modifiers: Optional[OpacityModifiers] = Field(
        default=None,
        description="Add location and chronicity details for the consolidation."
    )

class GroundGlass(BaseModel):
    """
    Hazy increased lung opacity that does not obscure vessels.
    """
    present: bool = Field(...,
    description="Indicate if ground-glass opacity is present."
    )
    pattern: Optional[str] = Field(
        default=None,
        description="Describe the pattern of ground-glass opacity (e.g., reticular, crazy paving)."
    )
    modifiers: Optional[OpacityModifiers] = Field(
        default=None,
        description="Add location and chronicity details for the ground-glass finding."
    )

class AirspaceOpacity(BaseModel):
    """
    Combines airspace opacities like consolidation and ground-glass.
    """
    consolidation: Optional[Consolidation] = Field(
        default=None,
        description="Describe findings related to consolidation."
    )
    ground_glass: Optional[GroundGlass] = Field(
        default=None,
        description="Describe findings related to ground-glass opacities."
    )


In [19]:
# -------------------------------------------------------------------
# LUNG NODULES AND MASSES
# -------------------------------------------------------------------

class NoduleModifiers(BaseModel):
    """
    Attributes that describe contextual characteristics of a radiographic finding.
    """
    lung_lobe: Optional[Literal[
        "Right Upper", "Right Middle", "Right Lower",
        "Left Upper", "Left Lower"
    ]] = Field(
        default=None,
        description="Select the lung lobe where the finding is located."
    )

    calcification: Optional[Literal[
        "None", "Punctate", "Coarse", "Diffuse"
    ]] = Field(
        default=None,
        description="Describe the pattern of calcification within the finding."
    )
    size_mm: Optional[float] = Field(
        default=None,
        description="Enter the maximum diameter of the nodule in millimeters."
    )
    suspicious: Optional[bool] = Field(
        default=None,
        description="Indicate if the nodule is radiologically suspicious."
    )


class Nodule(BaseModel):
    """
    Small rounded opacity in the lung (<3 cm).
    """
    distribution: Optional[Literal[
        "Solitary", "Multiple", "Diffuse"
    ]] = Field(
        default=None,
        description="Describe the spatial distribution of nodules."
    )
    modifiers: Optional[NoduleModifiers] = Field(
        default=None,
        description="Add location and chronicity details for the nodule."
    )

class Mass(BaseModel):
    """
    Larger lung lesion (>3 cm) that may represent malignancy or organized pathology.
    """
    distribution: Optional[Literal[
        "Solitary", "Multiple",
    ]] = Field(
        default=None,
        description="Describe the spatial distribution of masses."
    )
    modifiers: Optional[NoduleModifiers] = Field(
        default=None,
        description="Add location and chronicity details for the mass."
    )

class LungNodules(BaseModel):
    """
    Groups subtypes of solid pulmonary lesions.
    """
    nodule: Optional[Nodule] = Field(
        default=None,
        description="Provide findings related to pulmonary nodules (<3 cm)."
    )
    mass: Optional[Mass] = Field(
        default=None,
        description="Provide findings related to pulmonary masses (>3 cm)."
    )

In [20]:
# -------------------------------------------------------------------
# ROOT MODEL: STRUCTURED CXR FINDINGS
# -------------------------------------------------------------------

class CXRExtraction(BaseModel):
    """
    Root model representing structured extraction of CXR findings.
    """
    airspace_opacity: Optional[AirspaceOpacity] = Field(
        default=None,
        description="Include any airspace findings such as consolidation or ground-glass."
    )
    lung_nodules: Optional[LungNodules] = Field(
        default=None,
        description="Include any detected nodules or masses in the lungs."
    )

It is time to test our complex response model.

In [21]:
sampl_cxr_report = """
Clinical History: 65-year-old male with fever, cough, and recent weight loss.

Findings:

There is a solitary, well-circumscribed pulmonary nodule measuring approximately 8 mm in the left upper lobe. The nodule is partially calcified and appears non-suspicious on this study.

There is a consolidative opacity involving the right lower lobe, consistent with lobar pneumonia. No cavitation or pleural effusion is identified. The consolidation demonstrates an acute pattern.

No pulmonary mass is identified.

Impression:
	•	Left upper lobe solitary pulmonary nodule, 8 mm, likely benign. Recommend follow-up imaging per Fleischner guidelines.
	•	Right lower lobe consolidation, consistent with pneumonia.
	•	No evidence of pulmonary mass.

"""

In [22]:
# Test prompts
test3_system_prompt = "You are an expert in extracting data from radiology reports with 20 years of experience."
test3_user_prompt = """
Extract data elements from the diagnostic radiology CXR report in the <report> tag:
    Guidelines:
    - Focus on findings at the time of scan, not previous ones.
    - Ignore irrelevant information.
    """

In [23]:
# Let's ask the model

# Creating the conversation for the model to pass report and instructions
messages3 = [
        {"role": "system", "content": test3_system_prompt},
        {"role": "user", "content": f"{test3_user_prompt} \n <report> {sampl_cxr_report} </report>"}
]

# Asking the model to extract the requested information
resp3 = patched_client.chat.completions.create(
        model=MODEL,
        response_model=CXRExtraction, # changing the repsponse model
        messages=messages3,
        temperature=TEMP,
        seed=SEED,
        max_retries=MAX_RETRIES,
).model_dump_json(indent=4)


rich.print('Output:', resp3)

Output: {
    "airspace_opacity": {
        "consolidation": {
            "present": true,
            "extent": "lobar",
            "modifiers": {
                "lung_lobe": "Right Lower",
                "chronicity": "Acute"
            }
        },
        "ground_glass": null
    },
    "lung_nodules": {
        "nodule": {
            "distribution": "Solitary",
            "modifiers": {
                "lung_lobe": "Left Upper",
                "calcification": "Punctate",
                "size_mm": 8.0,
                "suspicious": false
            }
        },
        "mass": null
    }
}

## Conclusion
This notebook demonstrated how to achieve more consistent and structured output from Large Language Models (LLMs) by leveraging the `instructor` and `pydantic` libraries within a Google Colab environment. We first illustrated the potential inconsistencies of standard LLM outputs when asked to extract structured data. Then, we showed how patching the OpenAI client with `instructor` and defining a `pydantic` `BaseModel` allows for validation and automatic retries, ensuring the LLM's response adheres to a specified schema. Finally, we explored how to handle more complex data extraction scenarios, such as analyzing a chest X-ray report, by using nested `pydantic` models to create a hierarchical representation of the findings. This approach significantly enhances the reliability and usability of LLM-generated data for downstream tasks.

**Authors**:
- [Ali Ganjizadeh, M.D](https://www.linkedin.com/in/magnooj)
- [Bradley J. Erickson, M.D., Ph.D.](https://www.linkedin.com/in/bradleyerickson/)

This notebook is a part of [MIDel.org](http://midel.org/). `MIDeL` is a website to help healthcare professionals and medical imaging scientists learn to apply deep learning methods to medical images.